# Introduction
***
Cet outil permet de concevoir un survolteur (boost converter) en prévisualisant l'efficacité à l'aide des caractéristiques des composantes choisies. Deux types de topologie seront étudiés, La topologie de base étant la suivante (hard switching):
![dcdc](https://github.com/guimauve007/MPPT-builder/blob/main/assets/dcdc_boost.png?raw=1)



# Entrées
***
Ici, on peut entrer les caractéristiques des composantes choisies pour le DCDC:

In [1]:
# Inductance
ind_value = 220 # en uH
rl = 12 # en mOhm


In [2]:
# Switching FET
rds_sw = 11 # en mOhm
cds_sw = 400 # en pF
tsw = 31 # en ns -> temps de transition on-off

In [3]:
# Rectifier FET
rds_rect = 13 # en mOhm

# Équations (topologie de base)
***
La section suivante présente les différentes relations que l'on peut retrouver dans la topologie de base.

## Lexique
Les différentes variables utilisées dans les équations sont définies ci-dessous:

$$D=Duty\ Cycle\ (\%)$$
$$\Delta i=Current\ Ripple\ (A)$$
$$f=Frequency\ (Hz)$$
$$P=Période\ (Secondes)$$
$$L=Inductor\ (H)$$

## Duty Cycle
Relation entre Vin & Vout:

$$D=-\frac{Vin}{Vout}+1$$





In [4]:
def duty_cyle(vin, vout):
    return -(vin/vout)+1

## Ripple Current
Variation de courant dans l'inductance:
$$ \Delta i=\frac{vin*D}{f*L*2} $$


In [5]:
def delta_i(vin, d, f, l):
    return (vin*d)/(f*l*2)

## Pertes de rectification
Au lieu d'utiliser une diode pour la rectification du courant de sortie, on utilise un Mosfet avec une faible résistance en conduction. La perte induite dans cette composante est calculée comme suit:

$$P_{loss}=(I_{out})^2*R_{DS}$$
$$=>P_{loss}=((1-D)*I_{MPP})^2*R_{DS}$$

In [6]:
def ploss_rectifier(d, impp, rds_rect):
    return (((1-d)*impp)**2)*rds_rect

## Pertes de commutation
La perte de commutation est causée par le temps de commutation du mosfet principal. Cette perte est illustrée dans la figure suivante:

![switching loss](https://github.com/guimauve007/MPPT-builder/blob/main/assets/Switching-loss-of-MOSFETs-under-hard-switching.png?raw=1)

Afin de simplifier les calculs, on considérera la moyenne de T_on et T_off, nommé T_sw, pour calculer l'aire de la zone d'énergie dissipée. Pour une période complète, l'énergie perdue est calculée comme suit:

$$E_{loss}=2*T_{SW}*I_{MPP}*V_{DS}$$

La puissance totale dissipée à une fréquence donnée devient donc:

$$P_{loss}=2*T_{SW}*I_{MPP}*V_{DS}*f$$


In [7]:
def ploss_switching(tsw, impp, vds_sw, f):
    return 2*tsw*impp*vds_sw*f

## Pertes ESR (inductance)
Pertes en DC de l'inductance principale par sa résistance interne:

$$P_{loss}=R_L*[(I_{MPP})^2+\frac{\Delta I^2}{3}]$$

In [8]:
def ploss_inductor(rl, impp, delta_i):
    return rl*((impp**2)*(delta_i**2)/3)

## Pertes de commutation (Cds)
Pertes de commutation lié au condensateur parallèle $C_{DS}$. Cette perte est causée par la décharge de celui-ci lorsque le Mosfet commute:

$$P_{loss}=\frac{1}{2}*C_{DS}*(V_{DS})^2*f$$

In [9]:
def ploss_cds(cds, vds):
    return (1/2)*cds*(vds**2)*f